In [41]:
import requests

In [45]:
# # Get first page of biosample ids and soil horizon
base_url = "https://api.microbiomedata.org/nmdcschema/"

In [49]:
# Define a function to get first page of results (we will use again for other requests, which is why we are defining a function)
def get_first_page_results(collection: str, filter: str, max_page_size: int, fields: str):
    og_url = f"https://api.microbiomedata.org/nmdcschema/{collection}?&filter={filter}&max_page_size={max_page_size}&projection={fields}" 
    resp = requests.get(og_url)
    data = resp.json()

    return data

https://api.microbiomedata.org/nmdcschema/biosample_set?&filter={"soil_horizon":{"$exists": true}}&max_page_size=100&projection=id,soil_horizon
{'resources': [{'id': 'nmdc:bsm-11-002vgm56', 'soil_horizon': 'O horizon'}, {'id': 'nmdc:bsm-11-00dkyf35', 'soil_horizon': 'M horizon'}, {'id': 'nmdc:bsm-11-00hrxp98', 'soil_horizon': 'O horizon'}, {'id': 'nmdc:bsm-11-00m15h97', 'soil_horizon': 'M horizon'}, {'id': 'nmdc:bsm-11-00yhef97', 'soil_horizon': 'M horizon'}, {'id': 'nmdc:bsm-11-01f6m423', 'soil_horizon': 'O horizon'}, {'id': 'nmdc:bsm-11-01g9wf51', 'soil_horizon': 'O horizon'}, {'id': 'nmdc:bsm-11-01vt2q72', 'soil_horizon': 'A horizon'}, {'id': 'nmdc:bsm-11-02kcw433', 'soil_horizon': 'M horizon'}, {'id': 'nmdc:bsm-11-02n85875', 'soil_horizon': 'M horizon'}, {'id': 'nmdc:bsm-11-02v78297', 'soil_horizon': 'M horizon'}, {'id': 'nmdc:bsm-11-034x5t48', 'soil_horizon': 'M horizon'}, {'id': 'nmdc:bsm-11-03tatq76', 'soil_horizon': 'O horizon'}, {'id': 'nmdc:bsm-11-03wdqa25', 'soil_horizon': '

In [51]:
# Define another function to get the rest of the results using the next_page_token by calling the first funtion to get
# the initial results and then using the next_page_token to get the other results. This function returns a list of the results

def get_next_results(collection: str, filter: str, max_page_size: int, fields: str):

    # Get initial results (before next_page_token is given in results)
    initial_data = get_first_page_results(collection, filter, max_page_size, fields)
    results = initial_data["resources"]
    next_page_token = initial_data["next_page_token"]
    result_list = []

    # append first page of results to an empty list
    for result in results:
        result_list.append(result)
    
    while True:
        url = f"https://api.microbiomedata.org/nmdcschema/{collection}?&filter={filter}&max_page_size={max_page_size}&page_token={next_page_token}&projection={fields}"
        response = requests.get(url)
        data_next = response.json()
        
        results = data_next.get("resources", [])
        result_list.extend(results)
        next_page_token = data_next.get("next_page_token")
    
        if not next_page_token:
            break

    return result_list

biosamples = get_next_results("biosample_set", '{"soil_horizon":{"$exists": true}}', 100, "id,soil_horizon")
print(len(biosamples))

https://api.microbiomedata.org/nmdcschema/biosample_set?&filter={"soil_horizon":{"$exists": true}}&max_page_size=100&projection=id,soil_horizon
4650


In [40]:
# Next step is to use the ids from the biosamples as the has_input to get the ids for the Pooling class.

# get a list of the biosample ids for the filter parameter
biosamp_ids = []
for biosample in biosamples:
    biosamp_ids.append(biosample["id"])

# Put in mongodb-like query for the filter paramater
filt = f'{{"has_input": {{"$in": {biosamp_ids}}}}}'

poolings = get_next_results("pooling_set", filt, 100, "has_input,id")
print(poolings)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)